In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from google.cloud import bigquery, storage
import warnings
import os

warnings.filterwarnings("ignore")

### Designing the Data Warehouse Schema
A recommended approach is to break down final_data into dimension and fact tables. For example:

- Fact Table: fact_listings

Contains keys referencing dimension tables.
Contains metrics such as price_y, availability_365, number_of_reviews, revenue (derived).

- Dimension Tables:

dim_host: Information about hosts (host_id, host_name, host_since, host_location).
dim_geography: Information about location (neighbourhood_cleansed, city, state, country, latitude, longitude).
dim_property: Information about listings (property_type, room_type, accommodates, bedrooms, beds).
dim_date: A date dimension for first_review, last_review.
Note: If you have all data in one table, you can create views or separate tables to follow a star schema.

Split the final_data into dimension and fact tables locally, then upload:

In [2]:
final_data = pd.read_csv("/Users/asr/Desktop/College/CapstoneProject/CapStoneProject_Group3/dataset/final_data_with_tfidf.csv")

# Dimension: dim_host
dim_host = final_data[['host_id', 'host_name', 'host_since', 'host_location']]

# Dimension: dim_geography
dim_geography = final_data[['listing_id', 'neighbourhood_cleansed', 'city', 'state', 'country', 'latitude', 'longitude']]

# Dimension: dim_property
dim_property = final_data[['listing_id', 'property_type', 'room_type', 'accommodates', 'bedrooms', 'beds']]

# Dimension: dim_date (we'll focus on last_review as a primary date field)
dim_date = final_data[['listing_id', 'last_review']]

dim_date['last_review'] = pd.to_datetime(dim_date['last_review'])
dim_date['review_year'] = dim_date['last_review'].dt.year
dim_date['review_month'] = dim_date['last_review'].dt.month
dim_date['review_day'] = dim_date['last_review'].dt.day

# Fact Table: fact_listings
fact_listings = final_data[['listing_id', 'host_id', 'price_y', 'number_of_reviews', 'reviews_per_month', 'availability_365']]

dim_host.to_csv("dim_host.csv", index=False)
dim_geography.to_csv("dim_geography.csv", index=False)
dim_property.to_csv("dim_property.csv", index=False)
dim_date.to_csv("dim_date.csv", index=False)
fact_listings.to_csv("fact_listings.csv", index=False)

The code below creates a star schema in BigQuery.

- Use a star schema for simplicity:

- Fact Table:
    fact_listings: Contains metrics like price, availability, and reviews.
- Dimension Tables:
    dim_hosts: Contains host-specific details.
    dim_properties: Contains property-related details like room_type, property_type, and neighbourhood.
    dim_reviews: Contains details of reviews such as review scores and dates.

Upload these CSVs to GCS

In [5]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/Users/asr/Desktop/College/CapstoneProject/CapStoneProject_Group3/capstoneproject-444019-7bad7df402b1.json"
client = storage.Client()
bucket = client.bucket("airbnb-data-ingestion")

for file_name in ["dim_host.csv", "dim_geography.csv", "dim_property.csv", "dim_date.csv", "fact_listings.csv"]:
    blob = bucket.blob(f"airbnb/{file_name}")
    blob.upload_from_filename(file_name)

Load these into BigQuery as separate tables


In [ ]:
from google.cloud import bigquery

bq_client = bigquery.Client()
dataset_id = "airbnb_dataset"

def load_csv_to_bq(table_name, schema, uri):
    table_id = f"{bq_client.project}.{dataset_id}.{table_name}"
    job_config = bigquery.LoadJobConfig(
        schema=schema,
        source_format=bigquery.SourceFormat.CSV,
        skip_leading_rows=1,
        write_disposition="WRITE_TRUNCATE",
    )

    load_job = bq_client.load_table_from_uri(uri, table_id, job_config=job_config)
    load_job.result()
    print(f"Loaded {table_name}.")

# Define schemas
dim_host_schema = [
    bigquery.SchemaField("host_id", "INTEGER"),
    bigquery.SchemaField("host_name", "STRING"),
    bigquery.SchemaField("host_since", "DATE"),
    bigquery.SchemaField("host_location", "STRING")
]

dim_geography_schema = [
    bigquery.SchemaField("listing_id", "INTEGER"),
    bigquery.SchemaField("neighbourhood_cleansed", "STRING"),
    bigquery.SchemaField("city", "STRING"),
    bigquery.SchemaField("state", "STRING"),
    bigquery.SchemaField("country", "STRING"),
    bigquery.SchemaField("latitude", "FLOAT"),
    bigquery.SchemaField("longitude", "FLOAT")
]

dim_property_schema = [
    bigquery.SchemaField("listing_id", "INTEGER"),
    bigquery.SchemaField("property_type", "STRING"),
    bigquery.SchemaField("room_type", "STRING"),
    bigquery.SchemaField("accommodates", "INTEGER"),
    bigquery.SchemaField("bedrooms", "FLOAT"),
    bigquery.SchemaField("beds", "FLOAT")
]

dim_date_schema = [
    bigquery.SchemaField("listing_id", "INTEGER"),
    bigquery.SchemaField("last_review", "DATE"),
    bigquery.SchemaField("review_year", "INTEGER"),
    bigquery.SchemaField("review_month", "INTEGER"),
    bigquery.SchemaField("review_day", "INTEGER")
]

fact_listings_schema = [
    bigquery.SchemaField("listing_id", "INTEGER"),
    bigquery.SchemaField("host_id", "INTEGER"),
    bigquery.SchemaField("price_y", "FLOAT"),
    bigquery.SchemaField("number_of_reviews", "INTEGER"),
    bigquery.SchemaField("reviews_per_month", "FLOAT"),
    bigquery.SchemaField("availability_365", "INTEGER"),
    bigquery.SchemaField("revenue", "FLOAT")
]

load_csv_to_bq("dim_host", dim_host_schema, "gs://airbnb-data-ingestion/airbnb/dim_host.csv")
load_csv_to_bq("dim_geography", dim_geography_schema, "gs://airbnb-data-ingestion/airbnb/dim_geography.csv")
load_csv_to_bq("dim_property", dim_property_schema, "gs://airbnb-data-ingestion/airbnb/dim_property.csv")
load_csv_to_bq("dim_date", dim_date_schema, "gs://airbnb-data-ingestion/airbnb/dim_date.csv")
load_csv_to_bq("fact_listings", fact_listings_schema, "gs://airbnb-data-ingestion/airbnb/fact_listings.csv")


DefaultCredentialsError: Your default credentials were not found. To set up Application Default Credentials, see https://cloud.google.com/docs/authentication/external/set-up-adc for more information.

In [11]:
import pandas as pd

# Load the CSV file
input_file = '/Users/asr/Desktop/College/CapstoneProject/CapStoneProject_Group3/notebooks/dim_date.csv'  # Replace with your input CSV file path
output_file = '/Users/asr/Desktop/College/CapstoneProject/CapStoneProject_Group3/notebooks/dim_date_no_duplicates.csv'  # Replace with your desired output CSV file path

# Read the CSV into a DataFrame
df = pd.read_csv(input_file)

# Remove duplicates
df_cleaned = df.drop_duplicates()

# Save the cleaned DataFrame to a new CSV
df_cleaned.to_csv(output_file, index=False)

print(f"Duplicates removed. Cleaned data saved to '{output_file}'.")

Duplicates removed. Cleaned data saved to '/Users/asr/Desktop/College/CapstoneProject/CapStoneProject_Group3/notebooks/dim_date_no_duplicates.csv'.
